In [ ]:
# Package installs, run on new instances if using Google Colab
!pip install rasterio
!pip install image

In [ ]:
# Imports (some unused)

import keras
import rasterio
from rasterio.plot import show
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import skimage
from pylab import imshow
import time
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.model_selection import train_test_split

import math

In [ ]:
# NDWI function
def ndwi_img(image):
  
  #ndwi = (B3-B8A)/(B3+B8A)

  #convert to float for division
  band3 = image[1].astype(float)
  band8a = image[4].astype(float)

  sub = np.subtract(band3, band8a)
  add = np.add(band3, band8a)
  ndwi = np.divide(sub, add, out=np.zeros_like(sub), where=add!=0)

  return ndwi

# rgb function
def rgb_img(image):
  rgb = np.dstack((image[2],image[1],image[0]))
  return rgb

In [ ]:
# Import training data (flood pixels)
f = open("floodpixels.txt")
fpixels = np.loadtxt(f)
f.close()

# Non flood pixels
# Temporarily change this because non flood pixels export isnt working as intended, this is to be selected
#f = open("nonfloodpixels.txt")
f = open("nonfloodpixels.txt")
nfpixels = np.loadtxt(f)
f.close()


# Prestuff #

# Flood pixels
f = open("pfloodpixels.txt")
pfpixels = np.loadtxt(f)
f.close()

# Non flood pixels
f = open("pnonfloodpixels.txt")
pnfpixels = np.loadtxt(f)
f.close()

# S1 stuff #
f = open("floodpixelsS1.txt")
fpixelsS1 = np.loadtxt(f)
f.close()

f = open("nonfloodpixelsS1.txt")
nfpixelsS1 = np.loadtxt(f)
f.close()

fpixels = np.vstack([fpixels, fpixelsS1])
nfpixels = np.vstack([nfpixels, nfpixelsS1])


print(len(fpixels))
print(nfpixels)

In [ ]:
# Set up flood pixel and non flood pixel images
ndwifp = ndwi_img(fpixels)
ndwinfp = ndwi_img(nfpixels)

ndwipfp = ndwi_img(pfpixels)
ndwipnfp = ndwi_img(pnfpixels)

ndwidiff_fp = ndwifp - ndwipfp
ndwidiff_nfp = ndwinfp - ndwipnfp

print(ndwifp)
print(ndwinfp)

In [ ]:
# Debugging
print(type(fpixels[1][1]))
maxf = np.finfo('d').max
print(maxf)

In [ ]:
# Define features
pixel_features = ["B2","B3","B4","B8","B8A","B11","ndwi","rgb"]#not being used
pixel_features_temp = ["B2","B3","B4","B8","B8A","B11","VH diff", "VV diff"] # need to add ndwi and rgb to output
image_features = ["flooded","nonflooded"]

# Target
pixel_target_names = ["no flood","flood"]
image_target_names = ["no flood","flood"]
target_id = [0,1]

In [ ]:
# Append property names, target names and target id, then convert to pd dataframe

# Convert to dataframe
df = pd.DataFrame(np.transpose(fpixels),columns=pixel_features_temp)


df["ndwi"] = ndwi_img(fpixels) - ndwi_img(pfpixels)
#rgb is 3d so the same method doesn't work
df["target"] = 1
df

In [ ]:
# Non flood pixels
dnf = pd.DataFrame(np.transpose(nfpixels),columns=pixel_features_temp)
dnf["ndwi"] = ndwi_img(nfpixels)  - ndwi_img(pnfpixels)
dnf["target"] = 0
dnf

In [ ]:
# Join dataframes
df_all = pd.concat([df,dnf], ignore_index=1)
df_all

In [ ]:
# Separate into targets (kind of redundant but for readability)
df_all_0 = df_all[df_all.target==0]
df_all_1 = df_all[df_all.target==1]

# Example plot of ndwi of flood pixels vs non flood pixels
plt.scatter(df_all_0["ndwi"],df_all_0["B2"], color="g", marker = "+")
plt.scatter(df_all_1["ndwi"],df_all_1["B2"], color="b", marker = ".")
plt.xlabel("ndwi")
plt.ylabel("B2")

In [ ]:
# Same scatter, other way around
plt.scatter(df_all_1["ndwi"],df_all_1["B2"], color="b", marker = ".")
plt.scatter(df_all_0["ndwi"],df_all_0["B2"], color="g", marker = "+")
plt.xlabel("ndwi")
plt.ylabel("B2")

In [ ]:
# Example plot of VV and VH differences
plt.scatter(df_all_0["VH diff"],df_all_0["VV diff"], color="g", marker = "+")
plt.scatter(df_all_1["VH diff"],df_all_1["VV diff"], color="b", marker = ".")
plt.xlabel("VH")
plt.ylabel("VV")

In [ ]:
# Example plot of VV and VH differences
plt.scatter(df_all_1["VH diff"],df_all_1["VV diff"], color="b", marker = ".")
plt.scatter(df_all_0["VH diff"],df_all_0["VV diff"], color="g", marker = "+")
plt.xlabel("VH")
plt.ylabel("VV")

In [ ]:
# Drop rows with all 0
df_all = df_all.loc[df_all.B2+df_all.B3+df_all.B4+df_all.B8+df_all.B8A+df_all.B11!=0].reset_index().drop(["index"], axis="columns")
df_all

In [ ]:
# Histograms
df_no_target = df_all.drop(["target"], axis="columns")
ncols = len(df_no_target.columns)
histarr = np.zeros(ncols)
i=0
for column in df_no_target:
  # Select column contents by column name using [] operator
  columnSeriesObj = df_all[column]
  print(column)
  hist = df_no_target[column].hist(bins=30)
  plt.show()
  i=i+1

In [ ]:
# Remove the target column from dataframe
X = df_all.drop(["target"], axis="columns")
#X = X.drop(["B2","B3","B4","B8","B8A","B11","ndwi"], axis="columns")

#X = X.drop(["ndwi"], axis="columns")
#X.sort_values(by=["VH diff", "VV diff"], ascending=False)
X

In [ ]:
# Debuuging: sorting them by size and redifining them, obviously not for functional use, just debugging
# Randomize order of each target section - dont run this cell if you want actual data

X = X.abs().sort_values(by=["ndwi"], ascending=False, ignore_index=True)

#X = X.abs().sort_values(by=["VH diff"], ascending=False, ignore_index=True)
#X = X.abs().sort_values(by=["VV diff"], ascending=False, ignore_index=True)

X1 = X.head(int(920/2))
# Plot
plt.scatter(X["ndwi"],X["ndwi"], color="g", marker = "+")
plt.scatter(X1["ndwi"],X1["ndwi"], color="b", marker = "+")


plt.xlabel("ndwi")
plt.ylabel("ndwi")

X = X.drop(["B2","B3","B4","B8","B8A","B11"], axis="columns")
X


In [ ]:
# Dataframe of targets
y = df_all.target
y

In [ ]:
# Set up training, test on size, train on (1-size) - vary somewhat
# smaller a, larger training size, should be more accurate but at some point,
# the amount of testing data becomes too little so a few misclassifications become
# a much larger proportion
a = 0.6
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=a)

print(len(X_train)) #(1-size) of total length
print(len(X_test))

In [ ]:
# Classifier, play with parameters to get different accuracies
clf = svm.SVC(gamma="auto")

#clf = RandomForestClassifier()

# Fitting 
clf.fit(X_train, y_train)

# Score
score = clf.score(X_test, y_test)

print(clf)

print(score*100,"%")

rs =  round(score*100, 3 - int(math.floor(math.log10(abs(score*100)))) - 1)
print(rs,"% (3sf)")